# 네이버 영화평 감정 분석
- 119 ~ 152


In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Twitter # 단어의 품사를 알려줌
from tqdm.notebook import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_test.txt'

In [ ]:
def read_data(filename):
  with open(filename, 'r', encoding = 'utf8') as f:

    # 한줄씩 나누고, 탭기준으로 나눠서 리스트에 넣음
    result = [line.split('\t') for line in f.read().splitlines()]

    # header 제외 , 각 칸의 제목
    result = result[1:] 

  return result

In [ ]:
# 로드
train_tmp = read_data(train_path)
test_tmp = read_data(test_path)

train_tmp[:5]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1']]

In [ ]:
# 한글 품사 태깅 할 객체 생성
twitter = Twitter()

def getNounAdjective(text):
  # 빈 리스트
  stems = []

  # 품사 기본형 담음
  tagged = twitter.pos(text, stem = True)

  # 품사 형태가 명사나 형용사인것들의 단어를 리스트에 추가
  for i in range(len(tagged)):
    if (tagged[i][1] == 'Noun') or (tagged[i][1] == 'Adjective'):
      stems.append(tagged[i][0])
  return " ".join(stems)

In [ ]:
# 함수 확인
getNounAdjective("홍길동이 길을 걸어간다")

'홍길동 길'

In [ ]:
def kor_movie():
  # 리뷰 데이터 저장 리스트
  train_x = []

  # 감성(0,1) 데이터 저장 리스트
  train_y = []

  for i in tqdm(range(len(train_tmp))):
    # 리뷰
    train_x.append(getNounAdjective(train_tmp[i][1]))

    # 감성
    train_y.append(int(train_tmp[i][2]))

  test_x = []
  test_y = []

  for i in tqdm(range(len(test_tmp))):
    test_x.append(getNounAdjective(test_tmp[i][1]))

    test_y.append(int(test_tmp[i][2]))
  return (train_x, train_y), (test_x, test_y)

In [ ]:
# 함수 호출
(X_train, y_train), (X_test, y_test) = kor_movie()

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
# 가장 많이 사용하는 단어 999개를 숫자로 변환
tokenizer = Tokenizer(num_words = 999)

# X_train의 데이터들의 빈도 파악
tokenizer.fit_on_texts(X_train)

# tokenizer 가 어떤 단어를 숫자로 변환 했는지 확인
tokenizer.word_index.items()

# X_train의 단어들을 숫자로 변환
X_train_token = tokenizer.texts_to_sequences(X_train)

# X_test의 단어들을 숫자로 변환
X_test_token = tokenizer.texts_to_sequences(X_test)

In [ ]:
#  i번쨰 줄의 글자수가 2 미만이면 저장
one_word_idx = [i for i in range(len(X_train_token)) if len(X_train_token[i]) < 2]

# 단어수가 2미만에 해당하지 않으면 저장
X_train_token = [token for i, token in enumerate(X_train_token) if i not in one_word_idx]


y_train = [text for i, text in enumerate(y_train) if i not in one_word_idx]

In [ ]:
# 각 영화 리뷰 문장들의 길이가 다름
# 40길이로 맞추고 길이가 짧은 문자열들은 0으로 채움

X_train_pad = sequence.pad_sequences(X_train_token, maxlen =40)

X_test_pad = sequence.pad_sequences(X_test_token, maxlen =40)

In [ ]:
# 모델 객체 새성
model = Sequential()

model.add(Embedding(input_dim = 1000, # 인코딩 할 숫자 종류 0~999개, 즉 1000개
                    output_dim = 1000, # 인코딩 한 결과의 칸의수가 1000칸이 필요
                    input_length = 40, # 입력 데이터의 개수 리뷰 하나당 40개의 숫자
                    embeddings_initializer = Constant(np.eye(1000))))

model.add(LSTM(units = 128))

model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 1000)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               578048    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,578,177
Trainable params: 1,578,177
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 컴파일 및 훈련
model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(learning_rate = 1e-3),
              metrics = ['acc'])

# y_train 타입변환
y_train = np.array(y_train, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)

# 학습
model.fit(X_train_pad, y_train,
          batch_size =1000,
          epochs = 30)

Epoch 1/30
127/127 [==============================] - 17s 101ms/step - loss: 0.4366 - acc: 0.7997
Epoch 2/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3915 - acc: 0.8208
Epoch 3/30
127/127 [==============================] - 13s 104ms/step - loss: 0.3780 - acc: 0.8270
Epoch 4/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3690 - acc: 0.8313
Epoch 5/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3627 - acc: 0.8338
Epoch 6/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3556 - acc: 0.8382
Epoch 7/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3478 - acc: 0.8420
Epoch 8/30
127/127 [==============================] - 13s 100ms/step - loss: 0.3399 - acc: 0.8458
Epoch 9/30
127/127 [==============================] - 13s 101ms/step - loss: 0.3310 - acc: 0.8504
Epoch 10/30
127/127 [==============================] - 13s 101ms/step - loss: 0.3230 - acc: 0.8544
Epoch 11/30
127/127

In [ ]:
# 모델 평가
model.evaluate(X_test_pad, y_test)

1563/1563 [==============================] - 11s 6ms/step - loss: 0.8434 - acc: 0.7801


[0.8433747887611389, 0.7801399827003479]

In [ ]:
# 새로운 리뷰 예측
my_test_data = [
                "너무 지루 했습니다. 한 시간이 일년처럼 느껴 졌습니다",
                "옆사람의 통화 소리에 영화에 집중이 안 돼요",
                "오랜만에 본 감동적인 영화 였습니다"
]

test_data_list = []

for my_data in my_test_data:
  # 명사와 형용사만 리턴
  text = getNounAdjective(my_data)

  # 추가
  test_data_list.append(text)

# 단어를 숫자로 변환
x_test_token = tokenizer.texts_to_sequences(test_data_list)

# 문자 길이를 40으로 맞춤
x_test_pad = pad_sequences(x_test_token, maxlen =40)

In [ ]:
# 예측
predict = model.predict(x_test_pad)

# 0.5 이상 1, 0
predict_num = np.where(predict>0.5, 1,0)

# 1이면 긍정, 아니면 부정 리턴
predict_str = ['긍정' if pre ==1 else "부정" for pre in predict_num] 
predict_str

['부정', '긍정', '긍정']